In [2]:
import pandas as pd
import numpy as np
from ds.helper_functions import connect
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')
import sys
import ds.mongo_apollo as mg
import re ## for strings

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
import datetime
from datetime import timedelta

import os
# pd.set_option('precision', 0)

conn = connect(db="DS_Claims", server="USLSACASQL2", engine=True)
data_dir = "Z:/temp_data/sql_push/current/"

import upz

In [123]:
conn = connect(server = 'USLSACASQL2', db='DS_Claims', engine=True)
query = """SELECT *  FROM tep_claims"""
tep_claims = pd.read_sql(query, conn)
print(tep_claims.columns)
tep_claims.head(2)

Index(['patient_id', 'claim_id', 'svc_date', 'code', 'npi', 'referring_npi', 'hcp_spec', 'referring_hcp_spec', 'code_category', 'zip3', 'code_type', 'family', 'category', 'sub_category', 'qty', 'days_supply', 'strength', 'admin', 'brand'], dtype='object')


,patient_id,claim_id,svc_date,code,npi,referring_npi,hcp_spec,referring_hcp_spec,code_category,zip3,code_type,family,category,sub_category,qty,days_supply,strength,admin,brand
0,1598269464,10420257594400950568,2022-06-22,E0500,1164626248,1164626248,ENDOCRINOLOGY,ENDOCRINOLOGY,diag_cd_2,381,dx_icd10,symptom,graves,None,NaN,None,None,None,None
1,1583705968,10420257915800301020,2022-07-06,J3241,1811299944,1891061396,None,OPHTHALMOLOGY,prc_cd,379,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None


In [124]:
max_svc_date = tep_claims['svc_date'].max()
max_svc_date

datetime.date(2024, 3, 31)

In [125]:
conn = connect(server = 'USLSACASQL2', db='DS_Claims', engine=True)
query = """SELECT *  FROM tep_claims"""
tep_claims = pd.read_sql(query, conn)

## Add end of claims as column
tep_claims['svc_date'] = pd.to_datetime(tep_claims['svc_date'])
tep_claims['end_data_collection'] = [tep_claims['svc_date'].max()]*len(tep_claims)

In [126]:
tep_claims.head(5)

,patient_id,claim_id,svc_date,code,npi,referring_npi,hcp_spec,referring_hcp_spec,code_category,zip3,code_type,family,category,sub_category,qty,days_supply,strength,admin,brand,end_data_collection
0,1598269464,10420257594400950568,2022-06-22,E0500,1164626248,1164626248,ENDOCRINOLOGY,ENDOCRINOLOGY,diag_cd_2,381,dx_icd10,symptom,graves,None,NaN,None,None,None,None,2024-03-31
1,1583705968,10420257915800301020,2022-07-06,J3241,1811299944,1891061396,None,OPHTHALMOLOGY,prc_cd,379,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
2,165796137,10420258358203897876,2022-07-11,H18513,1629184239,1821017351,OPHTHALMOLOGY,OPTOMETRY,diag_cd_4,232,None,None,None,None,NaN,None,None,None,None,2024-03-31
3,1422487086,10420258540702799164,2022-07-16,84443,1689611337,1689611337,ENDOCRINOLOGY,ENDOCRINOLOGY,prc_cd,926,None,None,None,None,NaN,None,None,None,None,2024-03-31
4,2794108791,10420258698400708047,2022-05-17,J3241,1427132265,None,None,None,prc_cd,972,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31


In [127]:
tep_test = tep_claims[tep_claims['code'].isin(['J3241'])]
tep_test.head(10)  # Display the first 10 rows (if applicable)


,patient_id,claim_id,svc_date,code,npi,referring_npi,hcp_spec,referring_hcp_spec,code_category,zip3,code_type,family,category,sub_category,qty,days_supply,strength,admin,brand,end_data_collection
1,1583705968,10420257915800301020,2022-07-06,J3241,1811299944,1891061396,None,OPHTHALMOLOGY,prc_cd,379,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
4,2794108791,10420258698400708047,2022-05-17,J3241,1427132265,None,None,None,prc_cd,972,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
32,2384952491,10420264337101449749,2022-09-19,J3241,1174693592,None,None,None,prc_cd,243,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
52,154543114,10420268981804318815,2022-12-08,J3241,1578693016,1053687004,None,OPHTHALMOLOGY,prc_cd,078,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
58,412211765,10420270534102852495,2022-12-27,J3241,1922554138,1154316255,MID-LEVEL,OPHTHALMOLOGY,prc_cd,773,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
78,792627025,10420276568901483598,2023-03-17,J3241,1013489079,1194747253,MID-LEVEL,ENDOCRINOLOGY,prc_cd,780,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
91,87126212,10420279681907691216,2023-05-02,J3241,1447676952,1487698163,None,OPHTHALMOLOGY,prc_cd,071,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
101,11725253335,10420283188101875537,2023-06-14,J3241,1184238735,1104827351,MID-LEVEL,OPHTHALMOLOGY,prc_cd,786,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
114,1265521515,10420285473305142047,2023-08-02,J3241,1154418218,None,None,None,prc_cd,372,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31
171,592381478,10420291967906286043,2024-02-12,J3241,1427132265,None,None,None,prc_cd,971,cpt_hcpcs,treatment,tepezza,tepezza_cpt,NaN,None,None,None,None,2024-03-31


In [128]:
tep_test['code_type'].unique

<bound method Series.unique of 1          cpt_hcpcs
4          cpt_hcpcs
32         cpt_hcpcs
52         cpt_hcpcs
58         cpt_hcpcs
             ...    
2979905    cpt_hcpcs
2979962    cpt_hcpcs
2979981    cpt_hcpcs
2979994    cpt_hcpcs
2980050    cpt_hcpcs
Name: code_type, Length: 65317, dtype: object>

In [129]:
tep_claims = tep_claims.loc[(tep_claims['code']=='J3241')] #Only Tep Procedures/claims from dx+proc+remit
print('There are '+ str(tep_claims['patient_id'].nunique())+' patients w tpp claims')

There are 9519 patients w tpp claims


In [130]:
claims = tep_claims  # Need to change this, This is not the part of the original code

In [131]:
# def getSteroidStrength(claims): 

## Get Unique NDCs: 
# steroidNames = claims.loc[claims['sub_category'].isin(['iv_steroid','oral_steroid'])].drop_duplicates(subset = ['code_desc','strength','ndc'])[['code','code_desc','strength','category','sub_category','ndc','brand','qty','days_supply','admin']].reset_index(drop = True)  ---Here, NDC and CODE_DESC is not available on the data but it's asking here, so chaging this line
steroidNames = claims.loc[claims['sub_category'].isin(['iv_steroid','oral_steroid'])].drop_duplicates(subset = ['strength'])[['code','strength','category','sub_category','brand','qty','days_supply','admin']].reset_index(drop = True)
print('There are '+ str(len(steroidNames))+ ' steroid types (NDCs & code descriptions)')


There are 0 steroid types (NDCs & code descriptions)


In [132]:
def getSteroidStrength(claims): 
    
    ## Get Unique NDCs: 
    steroidNames = claims.loc[claims['sub_category'].isin(['iv_steroid','oral_steroid'])].drop_duplicates(subset = ['code_desc','strength','ndc'])[['code','code_desc','strength','category','sub_category','ndc','brand','qty','days_supply','admin']].reset_index(drop = True)
    print('There are '+ str(len(steroidNames))+ ' steroid types (NDCs & code descriptions)')

    ### Break into pack, pills, solutions (to modify strings accordingly):
    pack = ['TABLET THERAPY PACK']
    pills = ['TABLET', 'TABLET DISPERSIBLE', 'SOLUTION RECONSTITUTED','TABLET ENTERICCOATED']
    solutions = ['SYRUP', 'SOLUTION','SUSPENSION','CONCENTRATE']
    
    ## Check that there are no other admin modes: 
    li1 = list(steroidNames['admin'].unique())
    li2 = ['TABLET THERAPY PACK','TABLET', 'TABLET DISPERSIBLE', 'SOLUTION RECONSTITUTED','TABLET ENTERICCOATED','SYRUP', 'SOLUTION','SUSPENSION','CONCENTRATE']
    print('Number of Missing steroids administration types (should be <=1): '+ str(len(list(set(li1) - set(li2)))))
    print('Missing steroids administration types should only contain "None": '+ str(list(set(li1) - set(li2))))
    
    ### Get Strength Num for packs
    steroid_pack = steroidNames.loc[(steroidNames['admin'].isin(pack))&(steroidNames['sub_category']=='oral_steroid')]
    ## Remove all characters after paranthesis (qty)
    steroid_pack['strength_num'] = steroid_pack['code_desc'].str.split('(').str[0]
    # Only keep numbers (strength):
    steroid_pack['strength_num'] = steroid_pack['strength_num'].map(lambda x: re.sub("[^0-9]", "", x))
    steroid_pack.drop_duplicates(subset = ['code_desc'])

    ### Get Strength Num for pills (MG)
    steroid_mg = steroidNames.loc[(steroidNames['admin'].isin(pills))&(steroidNames['sub_category']=='oral_steroid')]
    steroid_mg['strength_num'] = steroid_mg['strength'].str.split(' ').str[0]
    steroid_mg.drop_duplicates(subset = ['strength'])

    steroidNames2 = pd.concat([steroid_pack,steroid_mg],ignore_index = True) ## Merge packs & pills

    ### Get Strength Num for solutions: 
    steroid_sol = steroidNames.loc[(steroidNames['admin'].isin(solutions))&(steroidNames['sub_category']=='oral_steroid')]
    steroid_sol['mg'] = steroid_sol['strength'].str.split('/').str[0]
    steroid_sol['mg'] = steroid_sol['mg'].map(lambda x: re.sub("[^.0-9]", "", x))
    steroid_sol['mg'] = steroid_sol['mg'].astype(np.float64)
    steroid_sol['mL'] = steroid_sol['strength'].str.split('/').str[1] ## split MG/ML (obtain ML)
    steroid_sol['mL'] = steroid_sol['mL'].map(lambda x: re.sub("[^0-9]", "", x)) ## convert to number
    steroid_sol['mL'] = steroid_sol['mL'].replace(r'^\s*$', 1, regex=True) ## Replace empty string with 1 (per 1ML)
    steroid_sol['mL'] = steroid_sol['mL'].astype(np.float64)
    steroid_sol['strength_num'] = np.round(steroid_sol['mg']/steroid_sol['mL'],2)
    steroid_sol.drop_duplicates(subset = ['strength'])

    steroidNames2 = pd.concat([steroidNames2,steroid_sol.drop(columns=['mg','mL'])],ignore_index = True) ## Merge packs, pills & solution

    ### Get Strength Num for IV Steroids: 
    steroid_iv = steroidNames.loc[steroidNames['sub_category']=='iv_steroid']
    steroid_iv['strength_num'] = steroid_iv['code_desc'].map(lambda x: re.sub("[^.0-9]", "", x))
    steroid_iv['strength_num'] = steroid_iv['strength_num'].replace(r'^\s*$', np.nan, regex=True) ## Replace empty string with np.nan (strength)

    steroid_iv.drop_duplicates(subset = ['code_desc','code'])

    steroidNames2 = pd.concat([steroidNames2,steroid_iv], ignore_index = True) ## Merge packs, pills, solution, and IVs
    steroidNames2['strength_num'] = steroidNames2['strength_num'].astype(np.float64)

    return steroidNames2

In [133]:
def _tep_features():
    pathIn = 'Z:/temp_data/sql_push/current/'

    #### Read data: 
    ## patient&claims table for both Hub patients 
    pat_tz = pd.read_parquet(pathIn + 'tep_patients.parquet')
    claims_tz = pd.read_parquet(pathIn + 'tep_claims.parquet')

    ## hcp_tz = contains NPIs for HCPs prescribing TEP, matched_hub = contains flag for patients matched in the hub (from Lihong fuzzy match)
    hcp_tz = pd.read_parquet(pathIn + 'tep_detail_providers.parquet')

    ## surgeryTypes = spreadsheet where surgeries were categorized; steroidTypes = spreadsheet where steroids were categorized (IV/low/high)
#     surgeryTypes = pd.read_csv('Z:/Projects/Adhoc/Claudia/tep_patient_journey/input/augustData/surgeriesCodes_augustPat_v1.csv')
    surgeryTypes = pd.read_csv('Z:/Projects/Adhoc/Claudia/tep_patient_journey/input/ted_surgery_code_groups.csv')
    pat_tz['patient_id'] = (pat_tz['patient_id'].astype(np.int64)).astype(str)

    ## Get all unique NDCs for each type of steroid
    steroidNames = getSteroidStrength(claims_tz)

    ## Add steroid types & strength back to claims:
    claims_tz2 = pd.merge(claims_tz, steroidNames[['code_desc','strength','ndc','strength_num']],how = 'left',on = ['code_desc','strength','ndc'])
    print('There are '+ str(len(claims_tz2.loc[claims_tz2['sub_category'].isin(['iv_steroid', 'oral_steroid'])].drop_duplicates(subset =['code_desc','strength','ndc']))) + ' types of steroids (NDCs)')
    claims_tz2.sort_values(by = ['patient_id','svc_date'], ascending = True, inplace = True)

    ## Change qty & Days_supply for IV to 1: 
    claims_tz2.loc[(claims_tz2['sub_category']=='iv_steroid')&(claims_tz2['qty'].isna())&(claims_tz2['strength_num'].notna()),'qty'] =1
    claims_tz2.loc[(claims_tz2['sub_category']=='iv_steroid')&(claims_tz2['days_supply'].isna())&(claims_tz2['strength_num'].notna()),'days_supply'] =1

    ## Get overall dose per prescription & Dose per Day: 
    claims_tz2['qty'] = claims_tz2['qty'].astype(np.float64);    claims_tz2['days_supply'] = claims_tz2['days_supply'].astype(np.float64)
    claims_tz2['dose_prescription_overall'] = claims_tz2['strength_num']*claims_tz2['qty']
    claims_tz2['dose_prescription_day'] = claims_tz2['dose_prescription_overall']/claims_tz2['days_supply']

    ## Add Dose Type: 
    claims_tz2['dose_type'] = np.nan
    claims_tz2.loc[(claims_tz2['sub_category']=='oral_steroid')&(claims_tz2['strength_num']>=20),'dose_type'] = 'high oral' ## Keep high as >=20 mg
    claims_tz2.loc[(claims_tz2['sub_category']=='oral_steroid')&(claims_tz2['strength_num']<20),'dose_type'] = 'low oral'
    claims_tz2.loc[(claims_tz2['sub_category']=='iv_steroid'),'dose_type'] = 'IV'

    ## Add Surgery types to Claims: 
    li1 = list(claims_tz2.loc[claims_tz2['category'] == 'surgery','code_desc'].unique()) ## list of all the surgery types in Claims
    li2 = list(surgeryTypes['code_desc'].unique()) ## list of all surgery types in spreadsheet
    print('Check that there are no steroid codes in claims not accounted for in the spreadsheet: '+ str(len(list(set(li1) - set(li2)))))
    claims_tz3 = pd.merge(claims_tz2,surgeryTypes[['code_desc','surgery_type','surgery_type2']],on = ['code_desc'],how = 'left')
    
    ## Add Specialties and sub-specialties from Apollo
    claims_tz4 = getSpecialties_fromNPI(claims_tz3)

    ## Define first_tep from IQVIA vs Hub:
    pat_tz['first_TEP_hub'] = pd.to_datetime(pat_tz['hub_therapy_start_date']);
    pat_tz['tep_proc_first_date'] = pd.to_datetime(pat_tz['tep_proc_first_date']) ## already had this calculated from cloud (it's first tepezza from iqvia claims)

    ## First TEPEZZA is the IQVIA one, unless that's missing (then replace with first_tep_Hub if present):
    pat_tz['first_TEP'] = pat_tz['tep_proc_first_date']
    pat_tz.loc[pat_tz['tep_proc_first_date'].isna(),'first_TEP'] = pat_tz.loc[pat_tz['tep_proc_first_date'].isna(),['tep_proc_first_date','first_TEP_hub']].min(axis = 1)

    ## Also get the month and quarter of the 1st TEP: 
    pat_tz['first_TEP_month'] = pd.to_datetime(pat_tz['first_TEP']).dt.to_period('M')
    pat_tz['first_TEP_quarter'] = pd.to_datetime(pat_tz['first_TEP']).dt.quarter

    ## Add time from dx --> TEP: 
    pat_tz2,claims_tz5 = getTimeFromTEP(pat_tz,claims_tz4)
    print('Patients with TEPEZZA: '+ str(pat_tz2.loc[pat_tz2['first_TEP'].notna(),'patient_id'].nunique())+ ' from total ' + str(pat_tz2['patient_id'].nunique()))

    ## Add 1st TEP hub Date (Month): 
    pat_tz2['first_TEPHub_month'] = pd.to_datetime(pat_tz2['earliest_hub_tep']).dt.to_period('M')

    ####### Qualifying patients ###################################################################### 
    ## Getting patients with TEP/Hub before last claim: 
    pat_tz2['first_tepHub_before_claims_end'] = np.zeros((len(pat_tz2),1))
    lastMonthClaims = pd.to_datetime(claims_tz5['svc_date']).dt.to_period('M').max()
    pat_tz2.loc[pat_tz2['first_TEPHub_month']<lastMonthClaims+1,'first_tepHub_before_claims_end']=1
    print('First Month TEP: '+ str(claims_tz.loc[claims_tz['category']=='tepezza','svc_date'].min()))
    print('Last month claims: '+ str(pd.to_datetime(claims_tz['svc_date']).dt.to_period('M').max()))

    ## Getting patients with > 6 mo claims history: 
    pat_tz2['6mo_claims_history'] = np.zeros((len(pat_tz2),1))
    pat_tz2.loc[pat_tz2['mo_FirstClaimToHub']>6,'6mo_claims_history']=1

    ## Getting patients with last claim < 3 months before 1st TEP: 
    pat_tz2['active_3mo_before_tep'] = np.zeros((len(pat_tz2),1))
    pat_tz2.loc[pat_tz2['mo_LastClaimToHub'] < 3,'active_3mo_before_tep']=1

    print('=='*50); print();
    print('Patients TEP/hub before last claims month: '+ str(pat_tz2.loc[pat_tz2['first_tepHub_before_claims_end']==1,'patient_id'].nunique()))
    print('Patients with at least 6 mo claims: '+ str(pat_tz2.loc[pat_tz2['6mo_claims_history']==1,'patient_id'].nunique()))
    print('Patients active 3 mo before TEP: '+ str(pat_tz2.loc[pat_tz2['active_3mo_before_tep']==1,'patient_id'].nunique()))
    print();print('=='*50);

    print('Patients with TEPEZZA before last claim: '+ str(pat_tz2.loc[(pat_tz2['first_tepHub_before_claims_end']==1),'patient_id'].nunique())); 
    print('Patients with TEPEZZA before last claim & > 6mo claims: '+ str(pat_tz2.loc[(pat_tz2['first_tepHub_before_claims_end']==1) & (pat_tz2['6mo_claims_history']==1),'patient_id'].nunique()));
    print('Patients with TEPEZZA before last claim & > 6mo claims & last claim < 3 mo before TEP: '+ str(pat_tz2.loc[(pat_tz2['first_tepHub_before_claims_end']==1)&(pat_tz2['6mo_claims_history']==1)&(pat_tz2['active_3mo_before_tep']==1),'patient_id'].nunique()))
    print();print('=='*50);

    ## Qualifying patients = 1st TEP before last claim & >6 mo claims history & last claim <3 mo from 1st TEP - modified 4/27 (to include patients with 1st hub date/tepezza before end claims)
    pat_tz3 = pat_tz2.copy()
    pat_tz3.loc[(pat_tz3['first_tepHub_before_claims_end']==1)&(pat_tz3['6mo_claims_history']==1)&(pat_tz3['active_3mo_before_tep']==1),"qualifying_tep_patient"]=1
    pat_tz3.loc[pat_tz3["qualifying_tep_patient"].isna(),"qualifying_tep_patient"]=0
    print('There are '+ str(pat_tz3.loc[pat_tz3['qualifying_tep_patient']==1,'patient_id'].nunique())+
          ' qualifying patients from '+ str(pat_tz3.loc[pat_tz3['earliest_hub_tep'].notna(),'patient_id'].nunique())+' TEP/Hub patients')
    
    ## Adding Oculoplastic Label:'Oculoplastics' or 'Other Oculofacial and Plastics' 
    seen_oculo = claims_tz5[claims_tz5["sub_specialty"].isin(['Oculoplastics','Other Oculofacial and Plastics'])][["patient_id"]].drop_duplicates()
    seen_oculo["seen_oculo"] = 1
    pat_tz3 = pat_tz3.merge(seen_oculo,"left","patient_id")

    ## Only Claims pre-TEPEZZA: 
    claims_pre_tz = claims_tz5.loc[pd.to_datetime(claims_tz5['svc_date'])<=pd.to_datetime(claims_tz5['earliest_hub_tep'])]
    print('Patients with Claims Pre-Hub: '+ str(claims_pre_tz['patient_id'].nunique()))
    
    print(claims_pre_tz)
    ## Adding patients with low oral/high oral/IV steroids (pre-Hub)
    doses = claims_pre_tz[claims_pre_tz.dose_type.isin(['low oral','high oral','IV'])][["patient_id","dose_type"]].drop_duplicates() #.query("dose_type.isin(['low oral','high oral','IV'])")
    doses["dose"] = 1
    doses = doses.set_index(["patient_id","dose_type"]).unstack()
    doses.columns = doses.columns.to_flat_index()
    new_columns = []
    for x in doses.columns.to_flat_index():
        line = "".join(x)
        if "dose" in line:
            line = "steroid_" + line[4:].lower()
        line = line.replace(' ','_')
        new_columns.append(line)
    doses.columns = new_columns
    doses = doses.reset_index()
    pat_tz3 = pat_tz3.merge(doses,"left","patient_id")

    print()
    print('All patients with IV steroids *in Claims* (pre-TEP): '+ str(claims_pre_tz.loc[claims_pre_tz['dose_type']=='IV','patient_id'].nunique()))
    print('All patients with IV steroids *in Patient Table* (pre-TEP): '+ str(pat_tz3.loc[(pat_tz3['steroid_iv']==1),'patient_id'].nunique()))
    print('Qualifying patients with IV steroids (pre-TEP): '+ str(pat_tz3.loc[(pat_tz3['steroid_iv']==1)&(pat_tz3['qualifying_tep_patient']==1),'patient_id'].nunique())); print()
    print('All patients with IV steroids *in Patient Table* (pre-TEP): '+ str(pat_tz3.loc[(pat_tz3['steroid_high_oral']==1),'patient_id'].nunique()))
    print('All patients with high Oral steroids *in Claims* (pre-TEP): '+ str(claims_pre_tz.loc[claims_pre_tz['dose_type']=='high oral','patient_id'].nunique()))
    print('Qualifying patients with high Oral steroids (pre-TEP): '+ str(pat_tz3.loc[(pat_tz3['steroid_high_oral']== 1)&(pat_tz3['qualifying_tep_patient']==1),'patient_id'].nunique())); print()

    ## Adding patients with surgery claims:
    surgery = claims_pre_tz[(claims_pre_tz.surgery_type.notna())|(claims_pre_tz.surgery_type2.notna())][["patient_id"]].drop_duplicates() #.query("(surgery_type.notna())or(surgery_type2.notna())")
    surgery["had_surgery"] = 1
    pat_tz3 = pat_tz3.merge(surgery,"left","patient_id")
    print('All patients with Surgeries *in Claims* (pre-TEP): '+ str(claims_pre_tz.loc[claims_pre_tz['surgery_type'].notna(),'patient_id'].nunique()))
    print('All patients with Surgeries *in Patient Table* (pre-TEP): '+ str(pat_tz3.loc[(pat_tz3['had_surgery']==1),'patient_id'].nunique()))
    print('Qualifying patients with Surgeries (pre-TEP): '+ str(pat_tz3.loc[(pat_tz3['had_surgery']== 1)&(pat_tz3['qualifying_tep_patient']==1),'patient_id'].nunique()))
    print()

    ### Get columns necessary for tep_patient table 
    pat_tz3['tep_start_date'] = pat_tz3['first_TEP']
    colsKeep = ["patient_id","qualifying_tep_patient","tep_start_date","seen_oculo","steroid_iv","steroid_high_oral","steroid_low_oral","had_surgery",
               'earliest_hub_tep','mo_FirstTEDToHub','incident_prevalent_label','acute_chronic_label']

    return pat_tz3[colsKeep]

In [134]:
conn = connect(db='Tepro', server='USLSACASQL2')

query = """SELECT *
FROM [Tepro].[dbo].[Tep_patient]
"""
tep_hub = pd.read_sql(query,conn)

tep_hub['enrollment_date']=pd.to_datetime(tep_hub['enrollment_date'])
tep_hub['infusion_1_date']=pd.to_datetime(tep_hub['infusion_1_date'])
tep_hub['scheduled_start_date']=pd.to_datetime(tep_hub['scheduled_start_date'])

In [135]:
tep_hub.head(2)

,patient_id,product_name,enrollment_date,aging,aging_name,first_pef_upload_date,hcp_id,treating_hcp_id,prescribing_hcp_id,enrolling_hcp_id,icd_code,icd_code_description,tier,dob,age,gender,height,weight,weight_uom,state,has_hipaa_consent,is_hps_participant,pam_status,pam_status_reason,reimbursement_status,hub_status,hub_status_reason,hub_sub_status,hub_status_date,soc_name,asoc_name,sp,predictability,primary_pbm,primary_payer,primary_paytype,primary_paytype_details,secondary_pbm,secondary_payer,secondary_paytype,secondary_paytype_details,prior_authorization_required,scheduled_start_date,next_treatment_date,infusion_1_date,infusion_2_date,infusion_3_date,infusion_4_date,infusion_5_date,infusion_6_date,infusion_7_date,infusion_8_date,est_vials_to_date,therapy_stop_date,therapy_restart_date,therapy_course_num,additional_course_pam_status,additional_course_pam_status_reason,scheduled_resume_date,sam_territory_code,sam_territory_name,sam_name,pam_territory_code,pam_territory_name,pam_name,ras_territory_code,ras_territory_name,ras_name,rbm_territory_code,rbm_territory_name,rbm_name,cm_territory_code,cm_name,alignment_zip,alignment_zip_soc,is_disrupted_patient,last_completed_infusion_num,last_completed_infusion_date,latest_pef_missing_info,inc_matrix_call_report,inc_matrix_call_report_source,acute_chronic_label,veeva_id,primary_payor_mastered,primary_payor_type_mastered,secondary_payor_mastered,secondary_payor_type_mastered,date_added,Enterprise,EnterpriseSeg
0,PM-24609,TEPEZZA,2020-01-22,57,46-60 Days,2020-01-22,1609805522,1609805522,None,1821439985,E05.00,thyrotoxicosis without thyrotoxic crisis or storm,On-Label,1977-02-24,47,F,None,70,kgs,GA,1,1,Inactive Post-IV,Treatment Complete,Approved,Closed,None,Treatment Complete,None,PARAGON ATLANTA INFUSION CENTER,CRAWFORD PLASTIC SURGERY,None,N/A,None,Alliant Health Plan Simple Care,Commercial,None,None,None,None,None,None,2020-03-19,None,2020-03-19,2020-04-10,2020-05-14,2020-08-05,2020-08-28,2020-09-24,2020-10-23,2020-11-13,None,None,None,2,Inactive Post-IV,Treatment Complete,None,611406,Atlanta East GA,Claire Hester,631503,Atlanta North GA,Julie Cowan,None,None,None,750201,Atlanta GA North,Tammi Moore,680207,Denise Apland,30720,30720,0,8,2020-11-13,None,0,None,Acute (0-3 years),a413w0000010yKgAAI,Alliant Health Plans,Commercial,None,None,2023-05-08,None,None
1,PM-24610,TEPEZZA,2020-01-22,43,31-45 Days,2020-01-22,1023054350,1023054350,None,1265540827,E05.00,thyrotoxicosis without thyrotoxic crisis or storm,On-Label,1954-11-02,69,F,None,NaN,None,NY,1,1,Inactive Post-IV,Treatment Complete,Approved,Closed,None,Treatment Complete,None,GLOBAL NEURO AND SPINE INSTITUTE,THRIVEWELL INFUSION HOLBROOK,None,N/A,Medicare of Florida,Medicare FFS (NY) - National Government Servic...,Medicare FFS,None,United Healthcare,United Healthcare,Commercial,None,None,2020-03-05,2020-03-26,2020-03-05,2020-03-26,2020-04-16,2020-05-07,2020-05-28,2020-06-18,2020-07-09,2020-07-30,None,None,None,2,Inactive Post-IV,Treatment Complete,None,611503,Ocala FL,Tyler Baisley,631907,Orlando FL,Minerva Caquimbo,None,None,None,750205,Miami FL,Javier Alvarez,680204,Breanne Martinez,Blank,11741,0,8,2020-07-30,None,0,None,None,a413w0000010yKlAAI,Medicare FFS (NY) - National Government Servic...,Medicare FFS,UHC / UnitedHealthcare,Commercial,2023-05-08,None,None


In [136]:
conn = connect(db='Tepro_Veeva', server='USLSACASQL2')

query = """ SELECT a.* , ci.Account_MVN__c, ci.State_MVN__c, ci.Zip_MVN__c, ci.Address_MVN__c
FROM [Tepro_Veeva].[dbo].[veeva_ib_accounts] AS a 
JOIN [Tepro_Veeva].[dbo].[veeva_ib_contact_info] AS ci ON a.Id = ci.Account_MVN__c
JOIN [Tepro_Veeva].[dbo].[veeva_ib_record_type] AS rt ON a.RecordTypeId = rt.Id
WHERE rt.Name = 'Patient'
AND ci.Address_MVN__c IS NOT NULL
"""
veeva_contact = pd.read_sql(query,conn)

# get zip3
veeva_contact['zip3']=veeva_contact['Zip_MVN__c'].str[0:3]

veeva_contact.head(2)

,AccountType,Id,IsDeleted,MasterRecordId,Name,LastName,FirstName,Salutation,Type,RecordTypeId,ParentId,Phone,Fax,AccountNumber,Website,PhotoUrl,Sic,Industry,AnnualRevenue,NumberOfEmployees,Ownership,TickerSymbol,Description,Rating,Site,OwnerId,CreatedDate,CreatedById,LastModifiedDate,LastModifiedById,SystemModstamp,LastActivityDate,MayEdit,IsLocked,LastViewedDate,LastReferencedDate,IsExcludedFromRealign,IsActive,EffectiveDate,EndDate,PersonContactId,IsPersonAccount,PersonMailingStreet,PersonMailingCity,PersonMailingState,PersonMailingPostalCode,PersonMailingCountry,PersonMailingStateCode,PersonMailingCountryCode,PersonMailingLatitude,PersonMailingLongitude,PersonMailingGeocodeAccuracy,PersonOtherStreet,PersonOtherCity,PersonOtherState,PersonOtherPostalCode,PersonOtherCountry,PersonOtherStateCode,PersonOtherCountryCode,PersonOtherLatitude,PersonOtherLongitude,PersonOtherGeocodeAccuracy,PersonMobilePhone,PersonHomePhone,PersonOtherPhone,PersonAssistantPhone,PersonEmail,PersonTitle,PersonDepartment,PersonAssistantName,PersonBirthdate,PersonHasOptedOutOfEmail,PersonHasOptedOutOfFax,PersonDoNotCall,PersonLastCURequestDate,PersonLastCUUpdateDate,PersonEmailBouncedReason,PersonEmailBouncedDate,PersonIndividualId,Jigsaw,JigsawCompanyId,AccountSource,SicDesc,OperatingHoursId,ConnectionReceivedId,ConnectionSentId,External_ID_vod__c,Credentials_vod__c,Territory_vod__c,Exclude_from_Zip_to_Terr_Processing_vod__c,Group_Specialty_1_vod__c,Group_Specialty_2_vod__c,Specialty_1_vod__c,Specialty_2_vod__c,Formatted_Name_vod__c,Territory_Test_vod__c,Mobile_ID_vod__c,Gender_vod__c,ID_vod__c,Do_Not_Sync_Sales_Data_vod__c,ID2_vod__c,Preferred_Name_vod__c,Sample_Default_vod__c,Segmentations_vod__c,Restricted_Products_vod__c,Payer_Id_vod__c,Alternate_Name_vod__c,Do_Not_Call_vod__c,Beds__c,Spend_Amount__c,PDRP_Opt_Out_vod__c,Spend_Status_Value_vod__c,PDRP_Opt_Out_Date_vod__c,Spend_Status_vod__c,Enable_Restricted_Products_vod__c,Call_Reminder_vod__c,Account_Group_vod__c,Primary_Parent_vod__c,Color_vod__c,Middle_vod__c,Suffix_vod__c,ATL_Last_Update_Date_Time_vod__c,No_Orders_vod__c,Account_Identifier_vod__c,Approved_Email_Opt_Type_vod__c,Account_Search_FirstLast_vod__c,Account_Search_LastFirst_vod__c,Language_vod__c,Practice_at_Hospital_vod__c,Practice_Near_Hospital_vod__c,Do_Not_Create_Child_Account_vod__c,Total_MDs_DOs__c,AHA__c,Order_Type_vod__c,NPI_vod__c,ME__c,Speaker__c,Investigator_vod__c,Default_Order_Type_vod__c,Customer_Master_Status_vod__c,Tax_Status__c,Model__c,Offerings__c,Departments__c,Account_Type__c,CLM_Opt_Type_vod__c,Account_Search_Business_vod__c,Business_Professional_Person_vod__c,Master_Align_Id_vod__c,Hospital_Type_vod__c,Account_Class_vod__c,Furigana_vod__c,NET_External_Id__c,Total_Revenue_000__c,Net_Income_Loss_000__c,PMPM_Income_Loss_000__c,Commercial_Premiums_PMPM__c,Medical_Loss_Ratio__c,Medical_Expenses_PMPM__c,Commercial_Patient_Days_1000__c,HMO_Market_Shr__c,HMO__c,HMO_POS__c,PPO__c,PPO_POS__c,Medicare__c,Medicaid__c,NET_Account_Status__c,Signature_Page_Display_Name_vod__c,NET_TIN__c,NET_Account_Type__c,Career_Status_vod__c,Photo_vod__c,Business_Description__c,Regional_Strategy__c,Contracts_Process__c,Country_MVN__c,Gender_MVN__c,Marital_Status_MVN__c,External_ID_MVN__c,Center_of_Excellence_MVN__c,Horizon_Specialty_MVN__c,PROCYSBI_Aligned_MVN__c,Primary_Address_Line_1_MVN__c,Primary_Address_Line_2_MVN__c,Primary_City_MVN__c,Primary_Country_MVN__c,Primary_State_MVN__c,Primary_Zip_Code_MVN__c,Testing_Site_MVN__c,Treating_Cystinosis_Patients_MVN__c,X18_Digit_Account_ID__c,Target__c,KOL_vod__c,Last_Interaction_Date_MVN__c,Num_Patient_Leads_MVN__c,Total_Lives__c,Total_Physicians_Enrolled__c,HZN_TEPEZZA_Clinical_in_service__c,Num_Patients_MVN__c,YTD_Interactions_MVN__c,HCP_Objections_MVN__c,NET_Birth_Year__c,NET_Credentials_2__c,NET_Place_of_Employment__c,NET_Type_of_Practice__c,NET_Veeva_Mastered__c,Data_Analysis__c,Territories_Aligned_MVN__c,of_Primary_Addresses__c,HZN_Secondary__c,ACTIMMUNE_Aligned_MVN__c,Preferred_Testing_Lab_

In [137]:
pathIn = 'Z:/temp_data/sql_push/2024m04/'

#### Read data: 
## patient&claims table for both Hub patients 
pat_tz = pd.read_parquet(pathIn + 'tep_patients.parquet')
claims_tz = pd.read_parquet(pathIn + 'tep_claims.parquet')

In [138]:
claims_tz.head(2)


,patient_id,claim_id,svc_date,code,npi,referring_npi,hcp_spec,referring_hcp_spec,code_category,zip3,code_type,family,category,sub_category,qty,days_supply,strength,admin,brand
0,1039049743,10015145820100004872,2019-01-04,23155007001,1255491106,None,PCP,None,rx,918,ndc,treatment,methimazole,nan,180.0,90,5 MG,TABLET,METHIMAZOLE
1,347731940,10015230536300015306,2021-06-10,23155007001,1528455474,None,ENDOCRINOLOGY,None,rx,913,ndc,treatment,methimazole,nan,45.0,30,5 MG,TABLET,METHIMAZOLE


In [139]:
claims = claims_tz

In [140]:
master_code=pd.read_csv('Z:/cluster/infused_product/config/config_latest_v1.csv', encoding='latin1')
print(master_code.columns)

Index(['code', 'prod', 'code_desc', 'code_type', 'family', 'category', 'sub_category', 'ndc', 'definition', 'code_added'], dtype='object')


In [141]:
# claims_ = pd.merge(master_code, claims, on='code', how='inner')
selected_columns_masterCode = master_code[['code', 'code_desc', 'ndc']]

# Merging df1 and df2 on 'code' column
claims_ = pd.merge(selected_columns_masterCode, claims, on='code', how='inner')

In [142]:
(claims_.columns)

Index(['code', 'code_desc', 'ndc', 'patient_id', 'claim_id', 'svc_date', 'npi', 'referring_npi', 'hcp_spec', 'referring_hcp_spec', 'code_category', 'zip3', 'code_type', 'family', 'category', 'sub_category', 'qty', 'days_supply', 'strength', 'admin', 'brand'], dtype='object')

In [143]:
steroidNames = claims_.loc[claims_['sub_category'].isin(['iv_steroid','oral_steroid'])].drop_duplicates(subset = ['code_desc','strength','ndc'])[['code','code_desc','strength','category','sub_category','ndc','brand','qty','days_supply','admin']].reset_index(drop = True)
print('There are '+ str(len(steroidNames))+ ' steroid types (NDCs & code descriptions)')

There are 261 steroid types (NDCs & code descriptions)


In [144]:
### Break into pack, pills, solutions (to modify strings accordingly):
pack = ['TABLET THERAPY PACK']
pills = ['TABLET', 'TABLET DISPERSIBLE', 'SOLUTION RECONSTITUTED','TABLET ENTERICCOATED']
solutions = ['SYRUP', 'SOLUTION','SUSPENSION','CONCENTRATE']

## Check that there are no other admin modes: 
li1 = list(steroidNames['admin'].unique())
li2 = ['TABLET THERAPY PACK','TABLET', 'TABLET DISPERSIBLE', 'SOLUTION RECONSTITUTED','TABLET ENTERICCOATED','SYRUP', 'SOLUTION','SUSPENSION','CONCENTRATE']
print('Number of Missing steroids administration types (should be <=1): '+ str(len(list(set(li1) - set(li2)))))
print('Missing steroids administration types should only contain "None": '+ str(list(set(li1) - set(li2))))

Number of Missing steroids administration types (should be <=1): 1
Missing steroids administration types should only contain "None": [None]


In [145]:
### Get Strength Num for packs
steroid_pack = steroidNames.loc[(steroidNames['admin'].isin(pack))&(steroidNames['sub_category']=='oral_steroid')]
## Remove all characters after paranthesis (qty)
steroid_pack['strength_num'] = steroid_pack['code_desc'].str.split('(').str[0]
# Only keep numbers (strength):
steroid_pack['strength_num'] = steroid_pack['strength_num'].map(lambda x: re.sub("[^0-9]", "", x))
steroid_pack.drop_duplicates(subset = ['code_desc'])

,code,code_desc,strength,category,sub_category,ndc,brand,qty,days_supply,admin,strength_num
93,00009005604,methylprednisolone tab therapy pack 4 mg (21),N/A,steroid,oral_steroid,00009005604,MEDROL DOSEPAK,21.0,6,TABLET THERAPY PACK,4
154,00591505221,prednisone tab therapy pack 5 mg (21),N/A,steroid,oral_steroid,00591505221,PREDNISONE,21.0,6,TABLET THERAPY PACK,5
155,00591505243,prednisone tab therapy pack 5 mg (48),N/A,steroid,oral_steroid,00591505243,PREDNISONE,48.0,6,TABLET THERAPY PACK,5
161,00591544221,prednisone tab therapy pack 10 mg (21),N/A,steroid,oral_steroid,00591544221,PREDNISONE,21.0,6,TABLET THERAPY PACK,10
162,00591544243,prednisone tab therapy pack 10 mg (48),N/A,steroid,oral_steroid,00591544243,PREDNISONE,48.0,12,TABLET THERAPY PACK,10


In [146]:
### Get Strength Num for pills (MG)
steroid_mg = steroidNames.loc[(steroidNames['admin'].isin(pills))&(steroidNames['sub_category']=='oral_steroid')]
steroid_mg['strength_num'] = steroid_mg['strength'].str.split(' ').str[0]
steroid_mg.drop_duplicates(subset = ['strength'])

steroidNames2 = pd.concat([steroid_pack,steroid_mg],ignore_index = True) ## Merge packs & pills

### Get Strength Num for solutions: 
steroid_sol = steroidNames.loc[(steroidNames['admin'].isin(solutions))&(steroidNames['sub_category']=='oral_steroid')]
steroid_sol['mg'] = steroid_sol['strength'].str.split('/').str[0]
steroid_sol['mg'] = steroid_sol['mg'].map(lambda x: re.sub("[^.0-9]", "", x))
steroid_sol['mg'] = steroid_sol['mg'].astype(np.float64)
steroid_sol['mL'] = steroid_sol['strength'].str.split('/').str[1] ## split MG/ML (obtain ML)
steroid_sol['mL'] = steroid_sol['mL'].map(lambda x: re.sub("[^0-9]", "", x)) ## convert to number
steroid_sol['mL'] = steroid_sol['mL'].replace(r'^\s*$', 1, regex=True) ## Replace empty string with 1 (per 1ML)
steroid_sol['mL'] = steroid_sol['mL'].astype(np.float64)
steroid_sol['strength_num'] = np.round(steroid_sol['mg']/steroid_sol['mL'],2)
steroid_sol.drop_duplicates(subset = ['strength'])


,code,code_desc,strength,category,sub_category,ndc,brand,qty,days_supply,admin,mg,mL,strength_num
111,00054372144,prednisone conc 5 mg/ml,5 MG/ML,steroid,oral_steroid,00054372144,PREDNISONE INTENSOL,60.0,8,CONCENTRATE,5,1,5
112,00054372250,prednisone oral soln 5 mg/5ml,5 MG/5 ML,steroid,oral_steroid,00054372250,PREDNISONE,150.0,15,SOLUTION,5,5,1
126,00093611816,prednisolone syrup 15 mg/5ml,15 MG/5 ML,steroid,oral_steroid,00093611816,PREDNISOLONE,110.0,28,SYRUP,15,5,3
130,00121077308,prednisolone sod phosphate oral soln 10 mg/5ml...,10 MG/5 ML,steroid,oral_steroid,00121077308,PREDNISOLONE SODIUM PHOSP,50.0,56,SOLUTION,10,5,2
147,00178058208,prednisolone sodium phosphate oral soln 25 mg/...,25 MG/5 ML,steroid,oral_steroid,00178058208,PREDNISOLONE SODIUM PHOSP,100.0,10,SOLUTION,25,5,5
195,13925016604,prednisolone sodium phosphate liq 6.7 mg/5ml,6.7 MG/5 ML,steroid,oral_steroid,13925016604,PREDNISOLONE SODIUM PHOSP,40.0,4,SOLUTION,7,5,1


In [147]:
steroidNames2 = pd.concat([steroidNames2,steroid_sol.drop(columns=['mg','mL'])],ignore_index = True) ## Merge packs, pills & solution

### Get Strength Num for IV Steroids: 
steroid_iv = steroidNames.loc[steroidNames['sub_category']=='iv_steroid']
steroid_iv['strength_num'] = steroid_iv['code_desc'].map(lambda x: re.sub("[^.0-9]", "", x))
steroid_iv['strength_num'] = steroid_iv['strength_num'].replace(r'^\s*$', np.nan, regex=True) ## Replace empty string with np.nan (strength)

steroid_iv.drop_duplicates(subset = ['code_desc','code'])

steroidNames2 = pd.concat([steroidNames2,steroid_iv], ignore_index = True) ## Merge packs, pills, solution, and IVs
steroidNames2['strength_num'] = steroidNames2['strength_num'].astype(np.float64)

In [148]:
steroidNames2

,code,code_desc,strength,category,sub_category,ndc,brand,qty,days_supply,admin,strength_num
0,00009005604,methylprednisolone tab therapy pack 4 mg (21),N/A,steroid,oral_steroid,00009005604,MEDROL DOSEPAK,21.0,6,TABLET THERAPY PACK,4
1,00591079021,methylprednisolone tab therapy pack 4 mg (21),N/A,steroid,oral_steroid,00591079021,METHYLPREDNISOLONE DOSE P,21.0,6,TABLET THERAPY PACK,4
2,00591505221,prednisone tab therapy pack 5 mg (21),N/A,steroid,oral_steroid,00591505221,PREDNISONE,21.0,6,TABLET THERAPY PACK,5
3,00591505243,prednisone tab therapy pack 5 mg (48),N/A,steroid,oral_steroid,00591505243,PREDNISONE,48.0,6,TABLET THERAPY PACK,5
4,00591544221,prednisone tab therapy pack 10 mg (21),N/A,steroid,oral_steroid,00591544221,PREDNISONE,21.0,6,TABLET THERAPY PACK,10
5,00591544243,prednisone tab therapy pack 10 mg (48),N/A,steroid,oral_steroid,00591544243,PREDNISONE,48.0,12,TABLET THERAPY PACK,10
6,00603459315,methylprednisolone tab therapy pack 4 mg (21),N/A,steroid,oral_steroid,00603459315,METHYLPREDNISOLONE DOSE P,21.0,6,TABLET THERAPY PACK,4
7,00603533715,prednisone tab therapy pack 5 mg (21),N/A,steroid,oral_steroid,00603533715,PREDNISONE,21.0,6,TABLET THERAPY PACK,5
8,00603533731,prednisone tab therapy pack 5 mg (48),N/A,steroid,oral_steroid,00603533731,PREDNISONE,30.0,30,TABLET THERAPY PACK,5
9,00603533815,prednisone tab therapy pack 10 mg (21),N/A,steroid,oral_steroid,00603533815,PREDNISONE,21.0,6,TABLET THERAPY PACK,10


In [149]:
inputDir='Z:/temp_data/sql_push/2024m04/'
claim_seq_df=pd.read_parquet(inputDir+'seq_df.parquet')#referral sequence from cloud
claim_seq_df[['prev_npi']]=claim_seq_df[['prev_npi']].astype(str)
claim_seq_df[['npi']]=claim_seq_df[['npi']].astype(str)
target_hcp=pd.read_excel('Z:/Projects/Adhoc/varun/ted_referral_prediction/inputs/20210226/TEPEZZA Target Score_update 20210216.xlsx')#will be updated soon. The target file
target_hcp[['NPI']]=target_hcp[['NPI']].astype(str)

ImportError: Pandas requires version '3.1.0' or newer of 'openpyxl' (version '3.0.10' currently installed).

In [ ]:
inputDir='Z:/temp_data/sql_push/2024m04/'
claim_seq_df=pd.read_parquet(inputDir+'seq_df.parquet')#referral sequence from cloud
claim_seq_df[['prev_npi']]=claim_seq_df[['prev_npi']].astype(str)
claim_seq_df[['npi']]=claim_seq_df[['npi']].astype(str)
target_hcp=pd.read_csv('Z:/Projects/Adhoc/varun/ted_referral_prediction/inputs/20210226/TEPEZZA Target Score_update 20210216_.csv', encoding='latin1')#will be updated soon. The target file
target_hcp[['NPI']]=target_hcp[['NPI']].astype(str)

In [ ]:
target_hcp.head(3)

,NPI,First Name,Last Name,Specialty,Sub Specialty,Address - Line 1,Address - Line 2,City,State,Zip Code,Territory ID,Territory,Account ID,Account Name,National Target Score,Territory Target Score,10-6 HCPs Type,HCPs Type
0,1376609362,MADHURA,TAMHANKAR,OPHTHALMOLOGY,Neuro-ophthalmology,51 N 39th St,NaN,Philadelphia,PA,19104,611204,Wilmington PA,216260,UPHS PRESBYTERIAN MEDICAL CENTER,10,10,Reason: TEPEZZA Active Writer,Treater
1,1063655181,MARK,KRAKAUER,OPHTHALMOLOGY,Oculoplastics,3401 N Broad St Ste 630 Fl 6,Parkinson Pavilion,Philadelphia,PA,19140,611204,Wilmington PA,218598,TEMPLE UNIVERSITY HOSPITAL INC,10,10,Reason: TEPEZZA Active Writer,Treater
2,1427235035,CESAR,BRICENO,OPHTHALMOLOGY,Oculoplastics,51 N 39th St,NaN,Philadelphia,PA,19104,611204,Wilmington PA,264791,INFUSION CENTER OF PENNSYLVANIA,10,10,Reason: TEPEZZA Active Writer,Treater


In [ ]:
data_dir = "Z:/temp_data/sql_push/2024m04/"

In [ ]:
df = pd.read_parquet(data_dir + "tep_patients.parquet")

df = df.merge(_tep_features(), "left", "patient_id")
df = df.merge(_hub_payer_info(), "left", "patient_id")

In [ ]:
pat_tz = pd.read_parquet(pathIn + 'tep_patients.parquet')
pat_tz['patient_id'] = (pat_tz['patient_id'].astype(np.int64)).astype(str)

In [ ]:
pat_tz['hub_therapy_start_date'].dtypes

dtype('O')

In [ ]:
pat_tz['first_TEP_hub'] = pd.to_datetime(pat_tz['hub_therapy_start_date'])
pat_tz['first_TEP_hub'].dtypes

dtype('<M8[ns]')

In [ ]:
pat_tz['tep_proc_first_date'] = pd.to_datetime(pat_tz['tep_proc_first_date'])
pat_tz['tep_proc_first_date'].dtypes

dtype('<M8[ns]')

In [ ]:
pat_tz['first_TEP'] = pat_tz['tep_proc_first_date']
pat_tz.loc[pat_tz['tep_proc_first_date'].isna(),'first_TEP'] = pat_tz.loc[pat_tz['tep_proc_first_date'].isna(),['tep_proc_first_date','first_TEP_hub']].min(axis = 1)

## Also get the month and quarter of the 1st TEP: 
pat_tz['first_TEP_month'] = pd.to_datetime(pat_tz['first_TEP']).dt.to_period('M')
pat_tz['first_TEP_quarter'] = pd.to_datetime(pat_tz['first_TEP']).dt.quarter



In [ ]:
pat_tz['first_TEP_quarter'].dtypes

dtype('float64')

In [ ]:
pat_tz['first_TEP'] = pd.to_datetime(pat_tz['first_TEP'])

In [ ]:
pat_tz['first_TEP'].dtypes

dtype('<M8[ns]')

In [ ]:
pat_tz['first_claim_date'].dtypes
pat_tz['first_claim_date'] = pd.to_datetime(pat_tz['first_claim_date'])
pat_tz['first_claim_date'].dtypes

dtype('<M8[ns]')

In [ ]:
pat_tz['first_claim_date'].dtypes

dtype('<M8[ns]')

In [ ]:
pat_tz['hub_enroll_date'] = pd.to_datetime(pat_tz['hub_enroll_date'])
pat_tz['hub_enroll_date'] .dtypes

dtype('<M8[ns]')

In [ ]:
pat_tz['last_claim_date'] = pd.to_datetime(pat_tz['last_claim_date'])
pat_tz['last_claim_date'].dtypes

dtype('<M8[ns]')

In [ ]:
# pat_tz['mo_FirstClaimToTep'] = np.round(((pat_tz['first_TEP'] - pat_tz['first_claim_date']).astype('timedelta64[D]'))/30,2)
pat_tz['mo_FirstClaimToTep'] = np.round(((pat_tz['first_TEP'] - pat_tz['first_claim_date']).astype('timedelta64[s]')) / (24 * 3600 * 30), 2)
pat_tz['mo_LastClaimToTep'] = np.round(((pat_tz['first_TEP'] - pat_tz['last_claim_date']).astype('timedelta64[s]')) / (24 * 3600 * 30), 2)



In [ ]:
pat_tz['earliest_hub_tep'] = pat_tz[['first_TEP','hub_enroll_date']].min(axis = 1); pat_tz['earliest_hub_tep'] = pd.to_datetime(pat_tz['earliest_hub_tep']);  


In [ ]:
print(pat_tz.columns)

Index(['patient_id', 'claims_count', 'first_claim_date', 'last_claim_date', 'tep_rx_count', 'eyedrop_proc_count', 'tep_rx_first_date', 'proptosis_dx_date', 'eyedrop_proc_last_date', 'eyedrop_proc_first_date', 'e0500_dx_date', 'strabismus_last_date', 'tep_proc_count', 'hashimotos_last_date', 'lid_retraction_count', 'diplopia_first_date', 'tep_rx_last_date', 'lid_retraction_last_date', 'surgery_proc_last_date', 'hashimotos_count', 'diplopia_count', 'hashimotos_first_date', 'tep_proc_first_date', 'surgery_proc_first_date', 'strabismus_count', 'lid_retraction_first_date', 'strabismus_first_date', 'surgery_count', 'graves_dx_date', 'tep_proc_last_date', 'diplopia_last_date', 'zip3', 'yob', 'gender', 'hub_enroll_date', 'hub_therapy_start_date', 'hub_hcp_npi', 'monotonic_row_id', 'date_created', 'graves_last_date', 'graves_first_date', 'graves_count', 'ted_broad_last_date', 'ted_broad_first_date', 'ted_broad_count', 'ted_common_last_date', 'ted_common_first_date', 'ted_common_count',
       '

In [ ]:
pathIn = 'Z:/temp_data/sql_push/2024m04/'
pat_tz = pd.read_parquet(pathIn + 'tep_patients.parquet')
claims_tz = pd.read_parquet(pathIn + 'tep_claims.parquet')

In [ ]:
print(pat_tz.columns)
print('-----------------------------------------------------------')
print(claims_tz.columns)

Index(['patient_id', 'claims_count', 'first_claim_date', 'last_claim_date', 'tep_rx_count', 'eyedrop_proc_count', 'tep_rx_first_date', 'proptosis_dx_date', 'eyedrop_proc_last_date', 'eyedrop_proc_first_date', 'e0500_dx_date', 'strabismus_last_date', 'tep_proc_count', 'hashimotos_last_date', 'lid_retraction_count', 'diplopia_first_date', 'tep_rx_last_date', 'lid_retraction_last_date', 'surgery_proc_last_date', 'hashimotos_count', 'diplopia_count', 'hashimotos_first_date', 'tep_proc_first_date', 'surgery_proc_first_date', 'strabismus_count', 'lid_retraction_first_date', 'strabismus_first_date', 'surgery_count', 'graves_dx_date', 'tep_proc_last_date', 'diplopia_last_date', 'zip3', 'yob', 'gender', 'hub_enroll_date', 'hub_therapy_start_date', 'hub_hcp_npi', 'monotonic_row_id', 'date_created', 'graves_last_date', 'graves_first_date', 'graves_count', 'ted_broad_last_date', 'ted_broad_first_date', 'ted_broad_count', 'ted_common_last_date', 'ted_common_first_date', 'ted_common_count',
       '

In [ ]:
# claims_tz_ = claims_tz[claims_tz['sub_category']=='diplopia']
# diplopia_first_date, lid_retraction_first_date, strabismus_first_date, 

# ted_symptom retired in v5 master code file

In [152]:
visual_disturbance_loss_discomfort_claims = claims_tz[claims_tz['sub_category'] == 'visual_disturbance/loss/discomfort']
df = visual_disturbance_loss_discomfort_claims.groupby('patient_id').agg(min_svc_date=('svc_date', 'min')).reset_index()
df = pd.merge(df, pat_tz, on='patient_id', how='inner')
pat_tz = df.rename(columns={'min_svc_date': 'visual_disturbance_loss_discomfort_first_date'})

In [153]:
pat_tz.head(1)

,patient_id,visual_disturbance_loss_discomfort_first_date,claims_count,first_claim_date,last_claim_date,tep_rx_count,eyedrop_proc_count,tep_rx_first_date,proptosis_dx_date,eyedrop_proc_last_date,eyedrop_proc_first_date,e0500_dx_date,strabismus_last_date,tep_proc_count,hashimotos_last_date,lid_retraction_count,diplopia_first_date,tep_rx_last_date,lid_retraction_last_date,surgery_proc_last_date,hashimotos_count,diplopia_count,hashimotos_first_date,tep_proc_first_date,surgery_proc_first_date,strabismus_count,lid_retraction_first_date,strabismus_first_date,surgery_count,graves_dx_date,tep_proc_last_date,diplopia_last_date,zip3,yob,gender,hub_enroll_date,hub_therapy_start_date,hub_hcp_npi,monotonic_row_id,date_created,graves_last_date,graves_first_date,graves_count,ted_broad_last_date,ted_broad_first_date,ted_broad_count,ted_common_last_date,ted_common_first_date,ted_common_count,ted_expanded_last_date,ted_expanded_first_date,ted_expanded_count,ted_common,ted_expanded,ted_broad,ted_profile,eye,graves_ted_dx_profile_date,ted_dx_date,ted_dx_profile,rx_data_through,dx_proc_data_through,hub_patient_id,match_desc,date_matched
0,1001041074,2019-05-29,41,2017-01-01,2024-01-24,0,0,None,2017-05-16,None,None,2019-06-13,None,0,None,0,None,None,None,2019-06-13,0,0,None,None,2019-06-13,0,None,None,1,2017-06-02,None,None,336,1967,F,20210607,None,None,1580547965620,2024-04-11,2024-01-24,2017-06-02,28,2019-05-29,2019-05-29,1,2024-01-24,2017-05-16,7,2019-05-29,2018-02-26,3,ted_common,ted_expanded,ted_broad,ted_common,2017-05-16,2017-06-02,2017-06-02,ted_common,2024-03-31,2024-03-31,None,None,None


In [154]:
pat_tz['visual_disturbance_loss_discomfort_first_date'].unique

<bound method Series.unique of 0      2019-05-29
1      2023-08-11
2      2019-10-21
3      2023-11-17
4      2021-05-25
          ...    
712    2023-11-27
713    2022-11-17
714    2022-05-26
715    2017-09-27
716    2018-01-12
Name: visual_disturbance_loss_discomfort_first_date, Length: 717, dtype: object>

In [155]:
pat_tz['proptosis_dx_date'] = pd.to_datetime(pat_tz['proptosis_dx_date'])
# for eye_symptom_dx_date
pat_tz['diplopia_first_date'] = pd.to_datetime(pat_tz['diplopia_first_date'])
pat_tz['lid_retraction_first_date'] = pd.to_datetime(pat_tz['lid_retraction_first_date'])
pat_tz['strabismus_first_date'] = pd.to_datetime(pat_tz['strabismus_first_date'])
pat_tz['visual_disturbance_loss_discomfort_first_date'] = pd.to_datetime(pat_tz['visual_disturbance_loss_discomfort_first_date'])

In [156]:
pat_tz['earliestEyeCode'] = pat_tz[['proptosis_dx_date','diplopia_first_date','lid_retraction_first_date','strabismus_first_date','visual_disturbance_loss_discomfort_first_date']].min(axis = 1)

In [157]:
pat_tz.head(1)

,patient_id,visual_disturbance_loss_discomfort_first_date,claims_count,first_claim_date,last_claim_date,tep_rx_count,eyedrop_proc_count,tep_rx_first_date,proptosis_dx_date,eyedrop_proc_last_date,eyedrop_proc_first_date,e0500_dx_date,strabismus_last_date,tep_proc_count,hashimotos_last_date,lid_retraction_count,diplopia_first_date,tep_rx_last_date,lid_retraction_last_date,surgery_proc_last_date,hashimotos_count,diplopia_count,hashimotos_first_date,tep_proc_first_date,surgery_proc_first_date,strabismus_count,lid_retraction_first_date,strabismus_first_date,surgery_count,graves_dx_date,tep_proc_last_date,diplopia_last_date,zip3,yob,gender,hub_enroll_date,hub_therapy_start_date,hub_hcp_npi,monotonic_row_id,date_created,graves_last_date,graves_first_date,graves_count,ted_broad_last_date,ted_broad_first_date,ted_broad_count,ted_common_last_date,ted_common_first_date,ted_common_count,ted_expanded_last_date,ted_expanded_first_date,ted_expanded_count,ted_common,ted_expanded,ted_broad,ted_profile,eye,graves_ted_dx_profile_date,ted_dx_date,ted_dx_profile,rx_data_through,dx_proc_data_through,hub_patient_id,match_desc,date_matched,earliestEyeCode
0,1001041074,2019-05-29,41,2017-01-01,2024-01-24,0,0,None,2017-05-16,None,None,2019-06-13,None,0,None,0,NaT,None,None,2019-06-13,0,0,None,None,2019-06-13,0,NaT,NaT,1,2017-06-02,None,None,336,1967,F,20210607,None,None,1580547965620,2024-04-11,2024-01-24,2017-06-02,28,2019-05-29,2019-05-29,1,2024-01-24,2017-05-16,7,2019-05-29,2018-02-26,3,ted_common,ted_expanded,ted_broad,ted_common,2017-05-16,2017-06-02,2017-06-02,ted_common,2024-03-31,2024-03-31,None,None,None,2017-05-16


In [159]:
def getSteroidStrength(claims): 
    print('here 4')
    ## Get Unique NDCs: 
    steroidNames = claims.loc[claims['sub_category'].isin(['iv_steroid','oral_steroid'])].drop_duplicates(subset = ['code_desc','strength','ndc'])[['code','code_desc','strength','category','sub_category','ndc','brand','qty','days_supply','admin']].reset_index(drop = True)
    print('There are '+ str(len(steroidNames))+ ' steroid types (NDCs & code descriptions)')
    print('here 5')
    ### Break into pack, pills, solutions (to modify strings accordingly):
    pack = ['TABLET THERAPY PACK']
    pills = ['TABLET', 'TABLET DISPERSIBLE', 'SOLUTION RECONSTITUTED','TABLET ENTERICCOATED']
    solutions = ['SYRUP', 'SOLUTION','SUSPENSION','CONCENTRATE']
    
    ## Check that there are no other admin modes: 
    li1 = list(steroidNames['admin'].unique())
    li2 = ['TABLET THERAPY PACK','TABLET', 'TABLET DISPERSIBLE', 'SOLUTION RECONSTITUTED','TABLET ENTERICCOATED','SYRUP', 'SOLUTION','SUSPENSION','CONCENTRATE']
    print('Number of Missing steroids administration types (should be <=1): '+ str(len(list(set(li1) - set(li2)))))
    print('Missing steroids administration types should only contain "None": '+ str(list(set(li1) - set(li2))))
    
    ### Get Strength Num for packs
    steroid_pack = steroidNames.loc[(steroidNames['admin'].isin(pack))&(steroidNames['sub_category']=='oral_steroid')]
    ## Remove all characters after paranthesis (qty)
    steroid_pack['strength_num'] = steroid_pack['code_desc'].str.split('(').str[0]
    # Only keep numbers (strength):
    steroid_pack['strength_num'] = steroid_pack['strength_num'].map(lambda x: re.sub("[^0-9]", "", x))
    steroid_pack.drop_duplicates(subset = ['code_desc'])

    ### Get Strength Num for pills (MG)
    steroid_mg = steroidNames.loc[(steroidNames['admin'].isin(pills))&(steroidNames['sub_category']=='oral_steroid')]
    steroid_mg['strength_num'] = steroid_mg['strength'].str.split(' ').str[0]
    steroid_mg.drop_duplicates(subset = ['strength'])

    steroidNames2 = pd.concat([steroid_pack,steroid_mg],ignore_index = True) ## Merge packs & pills

    ### Get Strength Num for solutions: 
    steroid_sol = steroidNames.loc[(steroidNames['admin'].isin(solutions))&(steroidNames['sub_category']=='oral_steroid')]
    steroid_sol['mg'] = steroid_sol['strength'].str.split('/').str[0]
    steroid_sol['mg'] = steroid_sol['mg'].map(lambda x: re.sub("[^.0-9]", "", x))
    steroid_sol['mg'] = steroid_sol['mg'].astype(np.float64)
    steroid_sol['mL'] = steroid_sol['strength'].str.split('/').str[1] ## split MG/ML (obtain ML)
    steroid_sol['mL'] = steroid_sol['mL'].map(lambda x: re.sub("[^0-9]", "", x)) ## convert to number
    steroid_sol['mL'] = steroid_sol['mL'].replace(r'^\s*$', 1, regex=True) ## Replace empty string with 1 (per 1ML)
    steroid_sol['mL'] = steroid_sol['mL'].astype(np.float64)
    steroid_sol['strength_num'] = np.round(steroid_sol['mg']/steroid_sol['mL'],2)
    steroid_sol.drop_duplicates(subset = ['strength'])

    steroidNames2 = pd.concat([steroidNames2,steroid_sol.drop(columns=['mg','mL'])],ignore_index = True) ## Merge packs, pills & solution

    ### Get Strength Num for IV Steroids: 
    steroid_iv = steroidNames.loc[steroidNames['sub_category']=='iv_steroid']
    steroid_iv['strength_num'] = steroid_iv['code_desc'].map(lambda x: re.sub("[^.0-9]", "", x))
    steroid_iv['strength_num'] = steroid_iv['strength_num'].replace(r'^\s*$', np.nan, regex=True) ## Replace empty string with np.nan (strength)

    steroid_iv.drop_duplicates(subset = ['code_desc','code'])

    steroidNames2 = pd.concat([steroidNames2,steroid_iv], ignore_index = True) ## Merge packs, pills, solution, and IVs
    steroidNames2['strength_num'] = steroidNames2['strength_num'].astype(np.float64)

    return steroidNames2

def getSpecialties_fromNPI(df_npi):
    ## Given df_npi get npi codes 
    npiCodes = list(df_npi['npi'].unique())

    ## Get npis: 
    dfApollo = mg.pull_mongo_data(npiCodes)
    print('Found '+ str(dfApollo['npi'].nunique()) + ' NPIs in Apollo from '+ str(len(npiCodes))+ ' NPIs in claims')

    ## Get columns of interest: 
    print('here here 1')
    cols = ['npi','specialty_group','sub_specialty','specialty']
    print('here here 2')
    df_npi = pd.merge(df_npi, dfApollo[cols], how = 'left', on = 'npi')
    print('here here 3')
    return df_npi

In [160]:
pathIn = 'Z:/temp_data/sql_push/2024m04/'

#### Read data: 
## patient&claims table for both Hub patients 
pat_tz = pd.read_parquet(pathIn + 'tep_patients.parquet')
claims_tz = pd.read_parquet(pathIn + 'tep_claims.parquet')
print('here 1')
master_code=pd.read_csv('Z:/cluster/infused_product/config/config_latest_v1.csv', encoding='latin1')
# print(master_code.columns)

selected_columns_masterCode = master_code[['code', 'code_desc', 'ndc']]
claims_tz = pd.merge(selected_columns_masterCode, claims_tz, on='code', how='inner')
print('here 2')
## hcp_tz = contains NPIs for HCPs prescribing TEP, matched_hub = contains flag for patients matched in the hub (from Lihong fuzzy match)
hcp_tz = pd.read_parquet(pathIn + 'tep_detail_providers.parquet')

## surgeryTypes = spreadsheet where surgeries were categorized; steroidTypes = spreadsheet where steroids were categorized (IV/low/high)
#     surgeryTypes = pd.read_csv('Z:/Projects/Adhoc/Claudia/tep_patient_journey/input/augustData/surgeriesCodes_augustPat_v1.csv')
surgeryTypes = pd.read_csv('Z:/Projects/Adhoc/Claudia/tep_patient_journey/input/ted_surgery_code_groups.csv')
pat_tz['patient_id'] = (pat_tz['patient_id'].astype(np.int64)).astype(str)
print('here 3')
## Get all unique NDCs for each type of steroid
steroidNames = getSteroidStrength(claims_tz)

## Add steroid types & strength back to claims:
claims_tz2 = pd.merge(claims_tz, steroidNames[['code_desc','strength','ndc','strength_num']],how = 'left',on = ['code_desc','strength','ndc'])
print('There are '+ str(len(claims_tz2.loc[claims_tz2['sub_category'].isin(['iv_steroid', 'oral_steroid'])].drop_duplicates(subset =['code_desc','strength','ndc']))) + ' types of steroids (NDCs)')
claims_tz2.sort_values(by = ['patient_id','svc_date'], ascending = True, inplace = True)

## Change qty & Days_supply for IV to 1: 
claims_tz2.loc[(claims_tz2['sub_category']=='iv_steroid')&(claims_tz2['qty'].isna())&(claims_tz2['strength_num'].notna()),'qty'] =1
claims_tz2.loc[(claims_tz2['sub_category']=='iv_steroid')&(claims_tz2['days_supply'].isna())&(claims_tz2['strength_num'].notna()),'days_supply'] =1

## Get overall dose per prescription & Dose per Day: 
claims_tz2['qty'] = claims_tz2['qty'].astype(np.float64);    claims_tz2['days_supply'] = claims_tz2['days_supply'].astype(np.float64)
claims_tz2['dose_prescription_overall'] = claims_tz2['strength_num']*claims_tz2['qty']
claims_tz2['dose_prescription_day'] = claims_tz2['dose_prescription_overall']/claims_tz2['days_supply']

## Add Dose Type: 
claims_tz2['dose_type'] = np.nan
claims_tz2.loc[(claims_tz2['sub_category']=='oral_steroid')&(claims_tz2['strength_num']>=20),'dose_type'] = 'high oral' ## Keep high as >=20 mg
claims_tz2.loc[(claims_tz2['sub_category']=='oral_steroid')&(claims_tz2['strength_num']<20),'dose_type'] = 'low oral'
claims_tz2.loc[(claims_tz2['sub_category']=='iv_steroid'),'dose_type'] = 'IV'

## Add Surgery types to Claims: 
li1 = list(claims_tz2.loc[claims_tz2['category'] == 'surgery','code_desc'].unique()) ## list of all the surgery types in Claims
li2 = list(surgeryTypes['code_desc'].unique()) ## list of all surgery types in spreadsheet
print('Check that there are no steroid codes in claims not accounted for in the spreadsheet: '+ str(len(list(set(li1) - set(li2)))))
claims_tz3 = pd.merge(claims_tz2,surgeryTypes[['code_desc','surgery_type','surgery_type2']],on = ['code_desc'],how = 'left')

## Add Specialties and sub-specialties from Apollo
claims_tz4 = getSpecialties_fromNPI(claims_tz3)

here 1
here 2
here 3
here 4
There are 261 steroid types (NDCs & code descriptions)
here 5
Number of Missing steroids administration types (should be <=1): 1
Missing steroids administration types should only contain "None": [None]
There are 261 types of steroids (NDCs)
Check that there are no steroid codes in claims not accounted for in the spreadsheet: 6
Found 103755 NPIs in Apollo from 106630 NPIs in claims
here here 1
here here 2
here here 3


In [161]:
claims_tz4.dtypes

code                          object
code_desc                     object
ndc                           object
patient_id                    object
claim_id                      object
svc_date                      object
npi                           object
referring_npi                 object
hcp_spec                      object
referring_hcp_spec            object
code_category                 object
zip3                          object
code_type                     object
family                        object
category                      object
sub_category                  object
qty                          float64
days_supply                  float64
strength                      object
admin                         object
brand                         object
strength_num                 float64
dose_prescription_overall    float64
dose_prescription_day        float64
dose_type                     object
surgery_type                  object
surgery_type2                 object
s

In [162]:
print(claims_tz4.columns)

Index(['code', 'code_desc', 'ndc', 'patient_id', 'claim_id', 'svc_date', 'npi', 'referring_npi', 'hcp_spec', 'referring_hcp_spec', 'code_category', 'zip3', 'code_type', 'family', 'category', 'sub_category', 'qty', 'days_supply', 'strength', 'admin', 'brand', 'strength_num', 'dose_prescription_overall', 'dose_prescription_day', 'dose_type', 'surgery_type', 'surgery_type2', 'specialty_group', 'sub_specialty', 'specialty'], dtype='object')


In [163]:
print('here here 4')

## Define first_tep from IQVIA vs Hub:
pat_tz['first_TEP_hub'] = pd.to_datetime(pat_tz['hub_therapy_start_date'])
print('here here 5')
pat_tz['tep_proc_first_date'] = pd.to_datetime(pat_tz['tep_proc_first_date']) ## already had this calculated from cloud (it's first tepezza from iqvia claims)
print('here here 6')
## First TEPEZZA is the IQVIA one, unless that's missing (then replace with first_tep_Hub if present):
pat_tz['first_TEP'] = pat_tz['tep_proc_first_date']
print('here here 7')
pat_tz.loc[pat_tz['tep_proc_first_date'].isna(),'first_TEP'] = pat_tz.loc[pat_tz['tep_proc_first_date'].isna(),['tep_proc_first_date','first_TEP_hub']].min(axis = 1)

## Also get the month and quarter of the 1st TEP: 
print('here here 8')
pat_tz['first_TEP_month'] = pd.to_datetime(pat_tz['first_TEP']).dt.to_period('M')
pat_tz['first_TEP_quarter'] = pd.to_datetime(pat_tz['first_TEP']).dt.quarter

here here 4
here here 5
here here 6
here here 7
here here 8


In [168]:
pat_tz = pat_tz
claims_tz = claims_tz4

In [171]:
pat_tz['first_TEP'] = pd.to_datetime(pat_tz['first_TEP']) 
print('here here 9.2')
pat_tz['first_claim_date'] = pd.to_datetime(pat_tz['first_claim_date'])  
pat_tz['hub_enroll_date'] = pd.to_datetime(pat_tz['hub_enroll_date'])
pat_tz['last_claim_date'] = pd.to_datetime(pat_tz['last_claim_date'])

here here 9.2


In [3]:
sys.path.append(('c:\\projects\\djosephs\\DS\\src\\processing\\'))
conn = connect(db='Tepro', server='USLSACASQL2')
query = """SELECT *  FROM [Tepro].[dbo].[Tep_patient]"""
tep_hub = pd.read_sql(query,conn)
tep_hub.rename(columns = {"patient_id":"hub_patient_id"}, inplace = True)